# Data

In [1]:
import pandas as pd


# Read the CSV file into a DataFrame
df = pd.read_csv("EEG/Copy of fused_transposed_GG1.csv")

# Display the first few rows
print(df.head())


              0             1             2             3             4  \
0  11799.284732  25973.866996   8188.203953 -10663.927873 -35497.006535   
1  11972.242531  25453.406626   8654.461343 -10467.009004 -35550.494260   
2  12159.527798  28133.336083  11770.026299 -10468.461867 -35849.918228   
3  12017.124834  29031.965617  11734.889356 -10659.748096 -35842.810374   
4  11815.936782  26442.828946   8583.606313 -10681.473992 -35537.530248   

             5             6            7  
0  3438.838236  26879.157350  9470.367070  
1  3470.309492  26851.307076  9466.455515  
2  3275.894019  26572.066733  9239.048867  
3  3213.420893  26560.130901  9208.002294  
4  3404.304791  26842.925172  9432.570271  


# Previous

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from functools import partial
from dualvit import dualvit_s

c:\Users\damic\anaconda3\envs\vision\lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: '[WinError 1920] Impossibile accedere al file'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:


# Assuming dualvit_s and simulate_image are already defined
# def dualvit_s(pretrained=False, **kwargs):
#     model = DualViT(
#         stem_hidden_dim=32,
#         embed_dims=[64, 128, 320, 448],
#         num_heads=[2, 4, 10, 14],
#         mlp_ratios=[8, 8, 4, 3, 2],
#         norm_layer=partial(nn.LayerNorm, eps=1e-6),
#         depths=[3, 4, 6, 3],
#         **kwargs)
#     return model

def simulate_image(batch_size=1, in_chans=3, height=224, width=224):
    return torch.rand(batch_size, in_chans, height, width)

# Parameters
num_classes = 4
batch_size = 16
num_epochs = 5
learning_rate = 0.001

# Instantiate model, loss function, and optimizer
model = dualvit_s(num_classes=num_classes)
model.train()  # Set to training mode
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Fake data loader
def fake_data_loader(num_batches, batch_size, in_chans, height, width, num_classes):
    for _ in range(num_batches):
        images = simulate_image(batch_size, in_chans, height, width)
        labels = torch.randint(0, num_classes, (batch_size,))
        yield images, labels

# Training loop
print("Starting Training...")
total_loss = 0
for images, labels in fake_data_loader(10, batch_size, 3, 224, 224, num_classes):
    optimizer.zero_grad()  # Clear gradients
    outputs = model(images)  # Forward pass
    
    # Extract logits if the model returns a tuple
    if isinstance(outputs, tuple):
        logits = outputs[0]
    else:
        logits = outputs

    loss = criterion(logits, labels)  # Compute loss
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights
    total_loss += loss.item()


# Testing loop
print("\nStarting Testing...")
model.eval()  # Set to evaluation mode
num_correct = 0
num_samples = 0
with torch.no_grad():
    for images, labels in fake_data_loader(5, batch_size, 3, 224, 224, num_classes):
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)  # Get class with highest probability
        num_correct += (predictions == labels).sum().item()
        num_samples += labels.size(0)

accuracy = num_correct / num_samples * 100
print(f"Testing Accuracy: {accuracy:.2f}%")


Starting Training...

Starting Testing...
Testing Accuracy: 21.25%
